In [10]:
spark.version

'3.1.3'

In [11]:
import os
import shutil
import pandas as pd
# import sh
from pyspark.sql.functions import *
#from pyspark.sql import functions as F
from pyspark.sql.types import *

In [12]:
pd.set_option("max_colwidth", 100)

#### Set-up COS functions for GCS

In [13]:
from google.cloud import storage

In [14]:
# List all files in given COS directory
def list_blobs(bucket_name, folder_name):
    gcs_client = storage.Client()
    bucket = gcs_client.bucket(bucket_name)
    blobs = list(bucket.list_blobs(prefix=folder_name))

    for blob in blobs:
        print(blob.name + '\t' + str(blob.size))

In [15]:
# List all files in given COS directory
def list_blobs_pd(bucket_name, folder_name):
    gcs_client = storage.Client()
    bucket = gcs_client.bucket(bucket_name)
    blobs = list(bucket.list_blobs(prefix=folder_name))

    blob_name = []
    blob_size = []
    
    for blob in blobs:
        blob_name.append(blob.name)
        blob_size.append(blob.size)

    blobs_df = pd.DataFrame(list(zip(blob_name, blob_size)), columns=['Name','Size'])

    blobs_df.style.format({"Size": "{:,.0f}"}) 
    
    return blobs_df

In [16]:
# Delete folder from COS bucket
def delete_folder(bucket_name, folder_name):
    gcs_client = storage.Client()
    bucket = gcs_client.bucket(bucket_name)
    blobs = list(bucket.list_blobs(prefix=folder_name))

    for blob in blobs:
        blob.delete()

In [19]:
# Reading data from open bucket, avaible to all students
bucket_read = 'msca-bdp-amazon'

# Saving results into individual bucket, students must update to their own bucket
bucket_write = 'msca-bdp-data-shared'

### Check source data in COS

In [20]:
list_blobs_pd(bucket_read, 'amazon_reviews')

,Name,Size
0,amazon_reviews/,0
1,amazon_reviews/amazon_reviews_us_Apparel_v1_00.tsv,1971061630
2,amazon_reviews/amazon_reviews_us_Automotive_v1_00.tsv,1350294084
3,amazon_reviews/amazon_reviews_us_Baby_v1_00.tsv,872274720
4,amazon_reviews/amazon_reviews_us_Beauty_v1_00.tsv,2152186111
5,amazon_reviews/amazon_reviews_us_Books_v1_00.tsv,6699171766
6,amazon_reviews/amazon_reviews_us_Books_v1_01.tsv,6703921864
7,amazon_reviews/amazon_reviews_us_Books_v1_02.tsv,3238702530
8,amazon_reviews/amazon_reviews_us_Camera_v1_00.tsv,1100169988
9,amazon_reviews/amazon_reviews_us_Digital_Ebook_Purchase_v1_00.tsv,6696964946


In [24]:
path = "gs://msca-bdp-amazon/amazon_reviews/amazon*.tsv"

In [25]:
amazon_df = spark.read.csv(path, header='true', inferSchema='true', sep='\t', quote='"')

In [26]:
spark.conf.set("spark.sql.repl.eagerEval.enabled",True)

In [28]:
amazon_df.printSchema()

root
 |-- marketplace: string (nullable = true)
 |-- customer_id: integer (nullable = true)
 |-- review_id: string (nullable = true)
 |-- product_id: string (nullable = true)
 |-- product_parent: integer (nullable = true)
 |-- product_title: string (nullable = true)
 |-- product_category: string (nullable = true)
 |-- star_rating: string (nullable = true)
 |-- helpful_votes: integer (nullable = true)
 |-- total_votes: integer (nullable = true)
 |-- vine: string (nullable = true)
 |-- verified_purchase: string (nullable = true)
 |-- review_headline: string (nullable = true)
 |-- review_body: string (nullable = true)
 |-- review_date: string (nullable = true)



## Question 1. Which product category gets most reviews?

### Answer: Digital_Ebook_Purchase with 13,718,761 reviews

To clean the data, I will only look at purchases that are verified

In [50]:
amazon_df = amazon_df.filter('verified_purchase = "Y"')

In [55]:
amazon_df.groupBy('product_category').agg(count('review_id')).orderBy('count(review_id)',ascending=False).show(1,False)

+----------------------+----------------+
|product_category      |count(review_id)|
+----------------------+----------------+
|Digital_Ebook_Purchase|13718761        |
+----------------------+----------------+
only showing top 1 row



## Question 2. Which product category gets highest / lowest review star ratings?

### Answer: Gift Card has the highest star ratings (4.7461), while Digital Software has the lowest star ratings (3.5430)

In [63]:
amazon_df = amazon_df.withColumn("star_rating", amazon_df["star_rating"].cast(DoubleType())).\
withColumn("review_date",amazon_df['review_date'].cast('date'))

In [59]:
df2 = amazon_df.groupBy('product_category').agg(avg('star_rating'))

In [60]:
df2.orderBy('avg(star_rating)',ascending = False).show(1,False)

+----------------+------------------+
|product_category|avg(star_rating)  |
+----------------+------------------+
|Gift Card       |4.7461078196439335|
+----------------+------------------+
only showing top 1 row



In [ ]:
df2.orderBy('avg(star_rating)',ascending = True).show(1,False)

+----------------+-----------------+
|product_category|avg(star_rating) |
+----------------+-----------------+
|Digital_Software|3.543014394580864|
+----------------+-----------------+
only showing top 1 row



## Question 3. Do you see seasonality in reviews? 

In [ ]:
Is every product category getting consistent number of reviews throughout the year?
Any product categories that are getting more reviews during certain times of the year?  If yes, which product categories and what times of the year?

In [72]:
amazon_df = amazon_df.withColumn("review_m_y", date_format(amazon_df["review_date"],"yyyy-MM")).\
                    withColumn("review_m", date_format(amazon_df["review_date"],"MM")).\
                    withColumn("review_y", date_format(amazon_df["review_date"],"yyyy"))

In [69]:
amazon_df.groupBy('review_m').agg(count('review_id'))

review_m_y,count(review_id)
null,7487
1995-11,1
1995-12,1
1996-02,1
1996-05,1
1996-06,8
1996-07,8
1996-08,2
1996-09,9
1996-10,14


In [73]:
%matplotlib inline
import pandas as pd
import matplotlib.pyplot as plt

In [ ]:
my_plot = amazon_df.plot(kind='line',x='review_m', y='review_y', legend=None, title="Height by Gender")